## Using the USGS Hydro Network-Linked Data Index (NLDI) API
The USGS provides a web service that enables the user to trace up- or down-stream from a given location and identify all the NHD catchments along that route. It's quite a useful tool for looking at hydologically connected areas, but we need to master the API to use this great resource. 

Below is an example of this API in action to help move you along if you want to use this in your work. 

#### Links:
* https://owi.usgs.gov/blog/nldi-intro/
* https://github.com/ACWI-SSWD/nldi-services - a link with good documentation on the API

In [ ]:
#Import the requests package to handle client-server requests
import requests

In [ ]:
#Set the service URL to a variable
nldiURL = 'https://cida.usgs.gov/nldi'

In [ ]:
#Set the API parameters
QTypes = {'DM':'Downstream Main',
          'UM':'Upstream Main',
          'DD':'Downstream with Diversions',
          'UT':'Upstream with Tributaries'}

In [ ]:
#User Values
f = 'huc12pp'       #SourceType
e = 'DM'            #QueryType - Here, we'll look Downstream along the Main channel
c = '180201251003'  #ComIDField - the NHD+ COMID of the start catchment
d = ''              #DistanceField - left blank to omit any distance limit

In [ ]:
#Service URLs
wqpURL = "{}/{}/{}/{}/wqp".format(nldiURL,f,c,e)
huc12ppURL = "{}/{}/{}/navigate/{}/huc12pp".format(nldiURL,f,c,e)
nhdURL = "{}{}/{}/navigate/{}".format(nldiURL,f,c,e)
basinURL = "{}/{}/{}/basin".format(nldiURL,f,c,e) 

In [ ]:
#Show an example of one of the URLs
print(basinURL)

In [ ]:
#Send the request and store the response
rawResp = requests.get(basinURL)

In [ ]:
#Convert the response to a JSON object (i.e. a Python dictionary)
j = rawResp.json()

In [ ]:
#Reveal the keys at the highest level 
j.keys()

In [ ]:
#Drill into the 'features' items
feats = j['features']
len(feats)

In [ ]:
#Explore the first feature item and the key's it has
feat = feats[0]
feat.keys()

In [ ]:
#Grab the "geom" item from the first feature; reveal it's datatype
geom = feat['geometry']
type(geom)

In [ ]:
#Explore the geom object...
geom.keys()

Turns out this `geom` object is a dictionary, but it has the format of a ["geoJSON"](https://geojson.org/) object. Python has a "geojson" package which allows us to handle these objects. 

In [ ]:
import geojson

_If the geojson object is missing in your environment install it by running the following in a new code cell:_
```
pip install geojson
```

In [ ]:
#Get the first feature and convert it into a muiltifeature object
m = geojson.MultiPolygon(feats[0]['geometry']['coordinates'])
type(m)

And now, we'll use geopandas to plot our multipolygon object...

In [ ]:
import geopandas as gpd
from geopandas import GeoSeries
from geopandas import GeoDataFrame

In [ ]:
#Create a geoseries from our multipolygon object
gSeries = GeoSeries(m)
type(gSeries)

In [ ]:
#Create geodataframe from our geoseries object
gdf = GeoDataFrame.from_features(feats)
type(gdf)

In [ ]:
#Tell jupyter to show plots "in-line"
%matplotlib inline

In [ ]:
#Polt the geodataframe
gdf.boundary.plot();

Yes, there are easier and more streamlined ways of doing this, but this reveals the basic constructs of APIs and how we can convert and use geometric features returned as server responses to our client requests.